In [1]:
import pandas as pd

from typing import Optional, Union
import pandas as pd, numpy as np, torch
from dataclasses import dataclass
from transformers import AutoTokenizer, AutoConfig
from transformers import EarlyStoppingCallback
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer, RobertaForMultipleChoice
import argparse
from transformers import get_polynomial_decay_schedule_with_warmup, TrainerCallback
import datasets
from datasets import Dataset

In [4]:
def get_label(row):
    label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    return label[-1]
    
df_train = pd.read_csv('dataset/train.csv').reset_index(drop = True)
df_train['label'] = df_train.apply(lambda x: get_label(x), axis = 1)

In [7]:
df_train.loc[df_train.index % 4 == 0,]

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,label
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0,1
8,441448,gpt-3.5-turbo-0613,vicuna-13b,"[""translate to russian the followig sentence ...","[""\u0411\u043e\u043b\u044c\u0448\u0438\u0435 \...","[""\u0411\u043e\u043b\u044c\u0448\u0438\u0435 \...",0,1,0,1
12,604575,pplx-70b-online,gpt-3.5-turbo-1106,"[""was the historical jesus a socialist"",""was h...","[""The question of whether the historical Jesus...","[""The historical Jesus lived in a time and pla...",0,1,0,1
16,887722,gpt-4-1106-preview,wizardlm-70b,"[""Write a grammatically correct sentence witho...","[""A lynx jumps quick.""]","[""The quick onyx goblin jumps over a lazy dwar...",1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
57460,4293918673,vicuna-13b,stablelm-tuned-alpha-7b,"[""Think step by step. If windows 10 would be s...","[""If Windows 10 were shipped on 3.5\"" floppies...","[""I'm sorry, I cannot answer this question wit...",0,0,1,2
57464,4294068603,gpt-4-0613,gpt-3.5-turbo-0613,"[""Sarah (female) has three brothers, her broth...","[""Sarah has two sisters.""]","[""Sarah has one sister.""]",0,1,0,1
57468,4294349169,palm-2,guanaco-33b,"[""Schreibe ein Bewerbungsanschreiben in deutsc...","[""I am trained to understand and respond only ...","[""Here is a possible Bewerbungsanschreiben in ...",0,1,0,1
57472,4294656694,gpt-4-0613,claude-1,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0,0


In [11]:
df_train.loc[df_train.index % 10 == 0,].reset_index(drop = True).to_csv('dataset/random_valid.csv')
df_train.loc[df_train.index % 10 != 0,].reset_index(drop = True).to_csv('dataset/random_train.csv')